Creates the example for poc.

In [1]:
%run sql_magics.ipynb

In [2]:
%run plpgsql_helpers.ipynb

CREATE FUNCTION


In [3]:
%%sql
DROP TABLE IF EXISTS base_raster;

CREATE TABLE base_raster(
    rid SERIAL primary key,
    rast raster
);

-- insert empty raster into the base_raster table

INSERT INTO base_raster(rast)
SELECT ST_AddBand( -- Returns a raster with the new band(s) of given type
  ST_MakeEmptyRaster( -- Returns an empty raster (having no bands)
      12, -- Raster width x (in pixels)
      8, -- Raster height y (in pixels)
      0, -- Upper left X coordinate
      0, -- Upper left Y coordinate
      1, -- X Cell Resolution (in SRID units - in this case meters)
      -1, -- Y Cell Resolution (in SRID units - in this case meters)
      0, -- X skew
      0, -- Y skew
      3794 -- SRID
    ),
    
    ARRAY [
      ROW(
        1,  -- band index
        '8BUI'::text,  -- 8BUI - 8-bit unsigned integer (https://postgis.net/docs/RT_ST_BandPixelType.html)
        3,  -- initial pixel value
        0  -- NODATA value
      )
    ]::addbandarg[] -- a composite type used as input into the ST_AddBand function (https://postgis.net/docs/addbandarg.html)
);

SELECT AddRasterConstraints('base_raster'::name, 'rast'::name);

NOTICE:  Adding SRID constraint
NOTICE:  Adding scale-X constraint
NOTICE:  Adding scale-Y constraint
NOTICE:  Adding blocksize-X constraint
NOTICE:  Adding blocksize-Y constraint
NOTICE:  Adding alignment constraint
NOTICE:  Adding number of bands constraint
NOTICE:  Adding pixel type constraint
NOTICE:  Adding nodata value constraint
NOTICE:  Adding out-of-database constraint
NOTICE:  Adding maximum extent constraint
 addrasterconstraints 
----------------------
 t
(1 row)



https://postgis.net/2014/09/26/tip_count_of_pixel_values/

In [4]:
%%sql

CREATE OR REPLACE FUNCTION myarray()
  RETURNS double precision[][] AS
\$func\$
DECLARE
    a double precision[][][8][12] := array_fill(0, ARRAY[8,12]);
BEGIN
	FOR i IN 1..8 LOOP
		FOR j IN 1..12 LOOP
			a[i][j]= (i-1)*12 + j;
		END LOOP;
	END LOOP;
   RETURN a;
END
\$func\$  LANGUAGE plpgsql;


UPDATE base_raster
SET rast = ST_SetValues(rast,1,1,1,myarray());

SELECT (gv).x, (gv).y, (gv).val, ST_AsText((gv).geom) geom
FROM (SELECT ST_PixelAsPolygons(rast) gv from base_raster)a


 x  | y | val |                   geom                   
----+---+-----+------------------------------------------
  1 | 1 |   1 | POLYGON((0 0,1 0,1 -1,0 -1,0 0))
  2 | 1 |   2 | POLYGON((1 0,2 0,2 -1,1 -1,1 0))
  3 | 1 |   3 | POLYGON((2 0,3 0,3 -1,2 -1,2 0))
  4 | 1 |   4 | POLYGON((3 0,4 0,4 -1,3 -1,3 0))
  5 | 1 |   5 | POLYGON((4 0,5 0,5 -1,4 -1,4 0))
  6 | 1 |   6 | POLYGON((5 0,6 0,6 -1,5 -1,5 0))
  7 | 1 |   7 | POLYGON((6 0,7 0,7 -1,6 -1,6 0))
  8 | 1 |   8 | POLYGON((7 0,8 0,8 -1,7 -1,7 0))
  9 | 1 |   9 | POLYGON((8 0,9 0,9 -1,8 -1,8 0))
 10 | 1 |  10 | POLYGON((9 0,10 0,10 -1,9 -1,9 0))
 11 | 1 |  11 | POLYGON((10 0,11 0,11 -1,10 -1,10 0))
 12 | 1 |  12 | POLYGON((11 0,12 0,12 -1,11 -1,11 0))
  1 | 2 |  13 | POLYGON((0 -1,1 -1,1 -2,0 -2,0 -1))
  2 | 2 |  14 | POLYGON((1 -1,2 -1,2 -2,1 -2,1 -1))
  3 | 2 |  15 | POLYGON((2 -1,3 -1,3 -2,2 -2,2 -1))
  4 | 2 |  16 | POLYGON((3 -1,4 -1,4 -2,3 -2,3 -1))
  5 | 2 |  17 | POLYGON((4 -1,5 -1,5 -2,4 -2,4 -1))
  6 | 2 |  18 | POLYGON(

In [5]:
%%sql

DROP TABLE IF EXISTS patch CASCADE;

CREATE TABLE patch(
    rid SERIAL primary key,
    rast raster
);

-- insert empty raster into the base_raster table

INSERT INTO patch(rast)
SELECT ST_AddBand( -- Returns a raster with the new band(s) of given type
  ST_MakeEmptyRaster( -- Returns an empty raster (having no bands)
      4, -- Raster width x (in pixels)
      4, -- Raster width y (in pixels)
      1.5, -- Upper left X coordinate
      -1.5, -- Upper left Y coordinate
      1, -- X Cell Resolution (in SRID units - in this case meters)
      -1, -- Y Cell Resolution (in SRID units - in this case meters)
      0, -- X skew
      0, -- Y skew
      3794 -- SRID
    ),
    
    ARRAY [
      ROW(
        1,  -- band index
        '8BUI'::text,  -- 8BUI - 8-bit unsigned integer (https://postgis.net/docs/RT_ST_BandPixelType.html)
        3,  -- initial pixel value
        0  -- NODATA value
      )
    ]::addbandarg[] -- a composite type used as input into the ST_AddBand function (https://postgis.net/docs/addbandarg.html)
);

SELECT AddRasterConstraints('patch'::name, 'rast'::name);

CREATE OR REPLACE FUNCTION myarray2()
  RETURNS double precision[][] AS
\$func\$
DECLARE
    a double precision[][][4][4] := array_fill(0, ARRAY[4,4]);
BEGIN
	FOR i IN 1..4 LOOP
		FOR j IN 1..4 LOOP
			a[i][j]= 100 + (i-1)*4 + j;
		END LOOP;
	END LOOP;
   RETURN a;
END
\$func\$  LANGUAGE plpgsql;


 -- ST_SetValue(raster rast, integer columnx, integer rowy, double precision newvalue);

UPDATE patch
SET rast = ST_SetValues(rast,1,1,1,myarray2());

SELECT (gv).x, (gv).y, (gv).val, ST_AsText((gv).geom) geom
FROM (SELECT ST_PixelAsPolygons(rast) gv from patch)a

NOTICE:  drop cascades to 2 other objects
DETAIL:  drop cascades to view patch_pixels
drop cascades to view patch_tiles
NOTICE:  Adding SRID constraint
NOTICE:  Adding scale-X constraint
NOTICE:  Adding scale-Y constraint
NOTICE:  Adding blocksize-X constraint
NOTICE:  Adding blocksize-Y constraint
NOTICE:  Adding alignment constraint
NOTICE:  Adding number of bands constraint
NOTICE:  Adding pixel type constraint
NOTICE:  Adding nodata value constraint
NOTICE:  Adding out-of-database constraint
NOTICE:  Adding maximum extent constraint
 x | y | val |                          geom                           
---+---+-----+---------------------------------------------------------
 1 | 1 | 101 | POLYGON((1.5 -1.5,2.5 -1.5,2.5 -2.5,1.5 -2.5,1.5 -1.5))
 2 | 1 | 102 | POLYGON((2.5 -1.5,3.5 -1.5,3.5 -2.5,2.5 -2.5,2.5 -1.5))
 3 | 1 | 103 | POLYGON((3.5 -1.5,4.5 -1.5,4.5 -2.5,3.5 -2.5,3.5 -1.5))
 4 | 1 | 104 | POLYGON((4.5 -1.5,5.5 -1.5,5.5 -2.5,4.5 -2.5,4.5 -1.5))
 1 | 2 | 105 | POLYGON((1.5 -

In [6]:
%%sql

-- tile the raster
DROP TABLE IF EXISTS dtm CASCADE;
SELECT (row_number() over ()) rid, rast into dtm from (SELECT ST_Tile(rast,3,3) rast from base_raster) a

SELECT 12


In [7]:
%%sql

SELECT visualize_tiles_and_pixels('dtm');
SELECT visualize_tiles_and_pixels('patch');

 visualize_tiles_and_pixels 
----------------------------
 
(1 row)

